In [63]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [64]:
df_customers  = pd.read_csv("Customers.csv")
df_products   = pd.read_csv("Products.csv")
df_transactions = pd.read_csv("Transactions.csv")


In [65]:
# 1. Merge Transactions with Customers
#    Join on CustomerID to get customer details in the transactions
merged_df1 = pd.merge(df_transactions, df_customers, on='CustomerID', how='left')

# 2. Merge the result with Products
#    Join on ProductID to get product details
merged_df = pd.merge(merged_df1, df_products, on='ProductID', how='left')

# Now, merged_df contains all the information from the three DataFrames:
merged_df.head(5)



,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [66]:
merged_df1 = merged_df.drop(columns=['ProductName','TransactionID','Price_x','Price_y','CustomerName'])

In [67]:
# merged_df1.info()

In [68]:

print(merged_df1.shape)
merged_df1.head(5)

(1000, 8)


,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Region,SignupDate,Category
0,C0199,P067,2024-08-25 12:38:23,1,300.68,Europe,2022-12-03,Electronics
1,C0146,P067,2024-05-27 22:23:54,1,300.68,Asia,2024-09-04,Electronics
2,C0127,P067,2024-04-25 07:38:55,1,300.68,Europe,2024-04-04,Electronics
3,C0087,P067,2024-03-26 22:55:37,2,601.36,South America,2024-04-11,Electronics
4,C0070,P067,2024-03-21 15:10:10,3,902.04,Europe,2022-03-15,Electronics


In [69]:
merged_df1['SignupMonth'] = pd.to_datetime(merged_df1['SignupDate']).dt.month

merged_df1['TransactionMonth'] = pd.to_datetime(merged_df1['TransactionDate']).dt.month

merged_df1 = merged_df1.drop(columns=['TransactionDate','SignupDate'])

merged_df1.head(5)

,CustomerID,ProductID,Quantity,TotalValue,Region,Category,SignupMonth,TransactionMonth
0,C0199,P067,1,300.68,Europe,Electronics,12,8
1,C0146,P067,1,300.68,Asia,Electronics,9,5
2,C0127,P067,1,300.68,Europe,Electronics,4,4
3,C0087,P067,2,601.36,South America,Electronics,4,3
4,C0070,P067,3,902.04,Europe,Electronics,3,3


In [70]:
customer_features = merged_df1.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'Region': lambda x: x.mode()[0],  # Get most frequent region
    'SignupMonth': 'first',
    'TransactionMonth': lambda x: x.mode()[0],
    'ProductID' : lambda x: x.mode()[0],
    'Category' : lambda x: x.mode()[0]
}).reset_index()

In [71]:
customer_features.shape

(199, 8)

In [72]:
customer_features

,CustomerID,TotalValue,Quantity,Region,SignupMonth,TransactionMonth,ProductID,Category
0,C0001,3354.52,12,South America,7,1,P022,Electronics
1,C0002,1862.74,10,Asia,2,2,P004,Clothing
2,C0003,2725.38,14,South America,3,6,P002,Home Decor
3,C0004,5354.88,23,South America,10,12,P008,Books
4,C0005,2034.24,7,Asia,8,3,P012,Electronics
...,...,...,...,...,...,...,...,...
194,C0196,4982.88,12,Europe,6,8,P079,Home Decor
195,C0197,1928.65,9,Europe,3,1,P013,Electronics
196,C0198,931.83,3,Europe,2,9,P064,Clothing
197,C0199,1979.28,9,Europe,12,8,P008,Electronics


In [73]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()

# Encode 'Region'
customer_features['Region_Encoded'] = le.fit_transform(customer_features['Region'])

# Encode 'Category'
customer_features['Category_Encoded'] = le.fit_transform(customer_features['Category'])

# Encode 'ProductID'
customer_features['ProductID_Encoded'] = le.fit_transform(customer_features['ProductID'])




In [74]:
customer_features = customer_features.drop(columns=['Region','ProductID','Category'])
customer_features


,CustomerID,TotalValue,Quantity,SignupMonth,TransactionMonth,Region_Encoded,Category_Encoded,ProductID_Encoded
0,C0001,3354.52,12,7,1,3,2,21
1,C0002,1862.74,10,2,2,0,1,3
2,C0003,2725.38,14,3,6,3,3,1
3,C0004,5354.88,23,10,12,3,0,7
4,C0005,2034.24,7,8,3,0,2,11
...,...,...,...,...,...,...,...,...
194,C0196,4982.88,12,6,8,1,3,62
195,C0197,1928.65,9,3,1,1,2,12
196,C0198,931.83,3,2,9,1,1,54
197,C0199,1979.28,9,12,8,1,2,7


In [75]:
customer_features

,CustomerID,TotalValue,Quantity,SignupMonth,TransactionMonth,Region_Encoded,Category_Encoded,ProductID_Encoded
0,C0001,3354.52,12,7,1,3,2,21
1,C0002,1862.74,10,2,2,0,1,3
2,C0003,2725.38,14,3,6,3,3,1
3,C0004,5354.88,23,10,12,3,0,7
4,C0005,2034.24,7,8,3,0,2,11
...,...,...,...,...,...,...,...,...
194,C0196,4982.88,12,6,8,1,3,62
195,C0197,1928.65,9,3,1,1,2,12
196,C0198,931.83,3,2,9,1,1,54
197,C0199,1979.28,9,12,8,1,2,7


In [76]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

scaler = MinMaxScaler()
customer_features[['TotalValue', 'Quantity']] = scaler.fit_transform(customer_features[['TotalValue', 'Quantity']])


In [77]:
customer_features

,CustomerID,TotalValue,Quantity,SignupMonth,TransactionMonth,Region_Encoded,Category_Encoded,ProductID_Encoded
0,C0001,0.308942,0.354839,7,1,3,2,21
1,C0002,0.168095,0.290323,2,2,0,1,3
2,C0003,0.249541,0.419355,3,6,3,3,1
3,C0004,0.497806,0.709677,10,12,3,0,7
4,C0005,0.184287,0.193548,8,3,0,2,11
...,...,...,...,...,...,...,...,...
194,C0196,0.462684,0.354839,6,8,1,3,62
195,C0197,0.174318,0.258065,3,1,1,2,12
196,C0198,0.080203,0.064516,2,9,1,1,54
197,C0199,0.179098,0.258065,12,8,1,2,7


In [78]:
feature_cols = ['TotalValue', 'Quantity', 'SignupMonth', 'TransactionMonth', 
                'Region_Encoded', 'Category_Encoded', 'ProductID_Encoded']
customer_features_extract = customer_features[feature_cols]


In [79]:
similarity_matrix = cosine_similarity(customer_features_extract)

In [80]:
top_lookalikes = {}
for i in range(20):  # First 20 customers
    customer_id = customer_features['CustomerID'][i]
    similarities = list(enumerate(similarity_matrix[i]))
    # Sort by similarity score (descending) and exclude self (i.e., customer itself)
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    top_3 = [(customer_features['CustomerID'][j], score) for j, score in sorted_similarities[1:4]]
    top_lookalikes[customer_id] = top_3

In [81]:
lookalike_df = pd.DataFrame([
    {'cust_id': cust_id, 'lookalikes': top_lookalikes[cust_id]} for cust_id in top_lookalikes
])
lookalike_df.to_csv('nishanthan_S_Lookalike.csv', index=False)